# Evaluate trained DeepParticleNet

In [ ]:
import os
import sys

from imgaug import augmenters as iaa
from keras.callbacks import EarlyStopping

# Add root directory to the python search path, if it is not already in there.
root_dir = os.path.abspath("..")
if root_dir not in sys.path:
    sys.path.append(root_dir)

# Import the DeepParticleNet modules.
from dpn.config import Config
from dpn.dataset import Dataset
from dpn.model import Model
from dpn.sizedistribution import SizeDistribution as PSD
from dpn import visualize

from external.CLR.clr_callback import CyclicLR

%matplotlib inline 

dataset_path = os.path.join(root_dir, "datasets")
dataset_name = "mpac"

## Setup configuration

In [ ]:
class ApplicationConfig(Config):
    NAME = "Application"
    BACKBONE = "resnet101"
    USE_PRETRAINED_WEIGHTS = "mpac"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    BATCH_SIZE = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 0.3
    MEAN_PIXEL = [60.2, 60.2, 60.2]
    
config = ApplicationConfig()

## Load Model

In [ ]:
model_dir = os.path.join(root_dir, "pretrained_weights")

# Create model in inference mode
model = Model(mode="inference", 
              model_dir=model_dir,
              config=config)

## Load Test Dataset

In [ ]:
# Inherit from the Dataset class.
class MpacDataset(Dataset):
    MONOCLASS = "sphere"  # The dataset only has one class.

# Test dataset.
dataset_name = "test_s06"
dataset = MpacDataset()
dataset.load_dataset(dataset_path, dataset_name)
dataset.prepare()

## Run Detection

In [ ]:
results = model.analyze_dataset(dataset)

In [ ]:
results.display_detection_image(2)